In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
import pygmt
from remit.earthvim import PolarityTimescale
from remit.utils.grid import make_dataarray, coeffs2map
from gprm.datasets import Seafloor

from remit.data.models import load_lcs, load_ocean_age_model, load_vis_model

from basis_models import load_vim_models

%load_ext autoreload
%autoreload 2


In [2]:
# A single cell to construct the VIM model
ocean = load_ocean_age_model()

vis = load_vis_model(match=(ocean.lon, ocean.lat, ocean.age))

pol = PolarityTimescale()
polarity_da = make_dataarray(ocean.lon, ocean.lat, pol.Interpolator(ocean.age))
polarity_da.data[np.isnan(ocean.age.data)] = np.nan



In [3]:

lmax = 140
cmax = 50
altitude = 0
showgravity = True

lcs = load_lcs(lmax=lmax)
lcs_rad = lcs.expand(a=lcs.r0).rad

model_dict = load_vim_models(
    [
     'GK07', 
    ], 
    altitude=altitude, lmax=lmax)


In [4]:
#grav_dir = '/mnt/c/Users/siwil/Data/GMTdata/VGG22/'
grav_dir = '../../../../Data/GMTdata/VGG22/'

def plot_mag_map(fig, rad, projection, region, cmax, showgravity=True):

    radi = pygmt.grdsample(grid=rad.to_xarray(), spacing='0.05d')
    pygmt.config(COLOR_FOREGROUND='red', COLOR_BACKGROUND='blue')
    pygmt.makecpt(cmap='polar', series='-{:f}/{:f}'.format(float(cmax), float(cmax)), reverse=True)
    radi.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid='./grids/_tmp.nc', transparency=10)

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    #'''
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='{:s}/grav_32.1.nc'.format(grav_dir),
                     shading='{:s}/grav_32.1.nc+a-45+nt2+m0'.format(grav_dir),
                     transparency=50)

    fig.plot(data='./gis/Muller_etal_AREPS_2016_C34.gmt', pen='2.0p,black')
    fig.plot(data='./gis/alternative_C34.gmt', pen='3.0,black,-', transparency=30)

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame=['afg','wSne'], region=region, projection=projection)
    
    pygmt.config(COLOR_FOREGROUND='red', COLOR_BACKGROUND='blue')
    pygmt.makecpt(cmap='polar', series='-{:f}/{:f}'.format(float(cmax), float(cmax)), reverse=True)
    with pygmt.config(FONT_ANNOT_PRIMARY='9p', FONT_LABEL='12p'):
        fig.colorbar(position='JBL+jBL+o0.7c/1.2c+w3.5c/0.33c+h+e', frame=['x+lBr [nT]'], box='+gwhite+p0.2p')


def plot_polarity(fig, polarity_da, projection, region, showgravity=True):
    pygmt.makecpt(cmap='hot', series='-1.5/1.5', reverse=True)
    polarity_da.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid = './grids/_tmp.nc',
                 interpolation='n', transparency=40)

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='{:s}/grav_32.1.nc'.format(grav_dir),
                     shading='{:s}/grav_32.1.nc+a-45+nt2+m0'.format(grav_dir),
                     transparency=50)
    
    fig.plot(data=Seafloor.MagneticPicks(load=False), style='c0.2c', fill='black', pen='1p,lightskyblue1', transparency=10, region=region, projection=projection)
    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame=['afg','WSne'], region=region, projection=projection)


def plot_seafloor_fabric(fig, projection, region, showgravity=True):

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='{:s}/grav_32.1.nc'.format(grav_dir),
                     shading='{:s}/grav_32.1.nc+a-45+nt2+m0'.format(grav_dir),
                     transparency=50)

    fig.plot(data=Seafloor.MagneticPicks(load=False), style='c0.05c', color='orange', transparency=20)
    fig.plot(data=Seafloor.SeafloorFabric(feature_type='FZ', load=False), pen='0.3p,black')

    fig.plot(data='./gis/CGMW_LipsSeamounts.shp', fill='darkred', pen='0.2p,black', transparency=20)
    fig.plot(data='./gis/Muller_etal_AREPS_2016_Ridges.gmt', pen='0.3p,dodgerblue')

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='darkgray', transparency=5)
    fig.basemap(frame=['afg','WSne'], region=region, projection=projection)


def PlotPanels(fig, projection, region, figsize, cmax, altitude=0, showgravity=True):
    
    (LCS_1, 
     model_1) = [lcs_rad, 
                 model_dict['GK07']['rad']]
    
    with fig.subplot(nrows=1, ncols=3, figsize=figsize, frame="lrtb", autolabel='+gwhite+pblack', margins='0.15i'):
        with fig.set_panel(panel=0, fixedlabel='Polarity'):
            plot_polarity(fig, polarity_da, projection, region, showgravity=showgravity)
        with fig.set_panel(panel=1, fixedlabel='LCS'):
            plot_mag_map(fig, LCS_1, projection=projection, region=region, cmax=cmax, showgravity=showgravity)
        with fig.set_panel(panel=2, fixedlabel='Sloping TRM'):
            plot_mag_map(fig, model_1, projection=projection, region=region, cmax=cmax, showgravity=showgravity)

            

In [5]:
projection='A-155/-45/?'
region = [-170,-50,-140,-28]
region_string = '{:f}/{:f}/{:f}/{:f}r'.format(region[0],region[1],region[2],region[3])


figsize=("12i", "3.8i")
showgravity=True

fig = pygmt.Figure()
with fig.subplot(nrows=1, ncols=3, figsize=figsize, frame="lrtb", autolabel='+gwhite+pblack', margins='0.15i'):
    with fig.set_panel(panel=0, fixedlabel='Polarity'):
        plot_polarity(fig, polarity_da, projection, region_string, showgravity=showgravity)
        
        with fig.inset(position="jTL+w2.5c+o0.2c", margin=0):
            # Create a figure in the inset using coast. This example uses the azimuthal
            # orthogonal projection centered at 47E, 20S. The land color is set to
            # "gray" and Madagascar is highlighted in "red3".
            fig.coast(
                region="g", projection="G-155/-45/?",
                land="gray", water="azure")
            rectangle = [[region[0], region[1], region[2], region[3]]]
            fig.plot(data=rectangle, style="r+s", fill="darkblue")
            fig.basemap(frame='g')
        
        with pygmt.config(MAP_TICK_PEN_PRIMARY='2p', FONT_ANNOT_PRIMARY='13p'):
            fig.basemap(map_scale="g-155/-33+w250k")#, box="+gwhite@30+p0.5p,gray30,solid+r3p")
            
    with fig.set_panel(panel=1, fixedlabel='LCS'):
        plot_mag_map(fig, lcs_rad, projection=projection, region=region_string, cmax=cmax, showgravity=showgravity)
    with fig.set_panel(panel=2, fixedlabel='GK07'):
        plot_mag_map(fig, model_dict['GK07']['rad'], projection=projection, region=region_string, cmax=cmax, showgravity=showgravity)
        
        

fig.savefig('./figures/Mismatch_SWPacific_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)


In [6]:
projection='A95/-20/?'
region = [85,-36,105,-15]
region_string = '{:f}/{:f}/{:f}/{:f}r'.format(region[0],region[1],region[2],region[3])


figsize=("12i", "4.8i")
showgravity=True

fig = pygmt.Figure()
with fig.subplot(nrows=1, ncols=3, figsize=figsize, frame="lrtb", autolabel='+gwhite+pblack', margins='0.15i'):
    with fig.set_panel(panel=0, fixedlabel='Polarity'):
        plot_polarity(fig, polarity_da, projection, region_string, showgravity=showgravity)
        
        with fig.inset(position="jTR+w2.5c+o0.1c/0.0c", margin=0):
            # Create a figure in the inset using coast. This example uses the azimuthal
            # orthogonal projection centered at 47E, 20S. The land color is set to
            # "gray" and Madagascar is highlighted in "red3".
            fig.coast(
                region="g", projection="G95/-20/?",
                land="gray", water="azure")
            rectangle = [[region[0], region[1], region[2], region[3]]]
            fig.plot(data=rectangle, style="r+s", fill="darkblue")
            fig.basemap(frame='g')
        
        with pygmt.config(MAP_TICK_PEN_PRIMARY='2p', FONT_ANNOT_PRIMARY='13p'):
            fig.basemap(map_scale="g102.5/-28+w250k")#, box="+gwhite@30+p0.5p,gray30,solid+r3p")
            
    with fig.set_panel(panel=1, fixedlabel='LCS'):
        plot_mag_map(fig, lcs_rad, projection=projection, region=region_string, cmax=cmax, showgravity=showgravity)
    with fig.set_panel(panel=2, fixedlabel='GK07'):
        plot_mag_map(fig, model_dict['GK07']['rad'], projection=projection, region=region_string, cmax=cmax, showgravity=showgravity)

fig.savefig('./figures/Mismatch_PAP_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)
